In [39]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
url="https://raw.githubusercontent.com/Arpit12gupta/ML-Dataset/master/wayfair_uniq.csv"
df = pd.read_csv(url)
df_new=df.iloc[:,[3,4,5,7,9,11,13,15,17,18,19]]

In [41]:
df_new['index'] = df_new.index

In [42]:
df_new.shape

(17209, 12)

In [43]:
df_new.head()

,product_name,product_name_link,manufacturer,price,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,from $38.02; $38.02,$69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,NaN,0
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,$73.99; $73.99,$99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,from $104.99; $104.99,$399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,$143.99; $143.99,$230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,$62.99; $62.99,$170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4


In [44]:
def get_title_from_index(index):
    return df_new[df_new.index == index]["product_name"].values[0]

def get_home(index):
    return df_new[df_new.index == index]["product_name_link"].values[0]

def get_index_from_title(title):
    return df_new[df_new.product_name == title]["index"].values[0]

In [45]:
print (df_new.columns)

Index(['product_name', 'product_name_link', 'manufacturer', 'price',
       'original_price', 'review_count', 'review_data', 'shipping_time',
       'product_descrip', 'product_data1', 'product_data2', 'index'],
      dtype='object')


In [46]:
df_new['original_price'] = df_new.original_price.str.replace('$', '')
df_new['original_price'] = df_new.original_price.str.replace(',', '').astype(float)

In [47]:
new_price = df_new["price"].str.split(";\s\$", n = 1, expand = True)
df_new["new_price"] = new_price[1]
df_new.drop(columns =["price"], inplace = True)

In [48]:
price_range = df_new[df_new["new_price"].str.contains("-",regex=True)]

In [49]:
price_range['new_price'] = (price_range['new_price'].str.split("\s-", n = 1, expand = True))[0]

In [50]:
df_new.loc[price_range.index] = price_range

In [51]:
df_new['new_price'] = df_new.new_price.str.replace(',', '').astype(float)

In [52]:
bins = [0,500,1000,1500,2000,2500,3000,3500,4000,4500,35000]
labels = ["<$500","$500-$1000","$1000-$1500","$1500-$2000","$2000-$2500","$2500-$3000","$3000-$3500","$3500-$4000","$4000-$4500",">$4500"]
df_new['price_bracket'] = pd.cut(df_new['new_price'], bins, labels=labels)

In [53]:
features = ["product_data1","product_data2","product_descrip","manufacturer"]

In [54]:
for feature in features:
    df_new[feature] = df_new[feature].fillna('')

In [55]:
def combine_features(row):
    try:
        return row['product_data1'] +" "+row['product_data2']+" "+row["product_descrip"]+" "+row["price_bracket"]+" "+row["manufacturer"]
    except:
        print("Error:", row)

In [56]:
df_new["combined_features"] = df_new.apply(combine_features,axis=1)

In [57]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,Switch Type: On/Off switch Short on space? Ma...
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,Switch Type: Foot Switch Maximum Wattage (per ...
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,Switch Type: Rotary socket Maximum Wattage (pe...
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,Switch Type: 4-Way Maximum Wattage (per Bulb):...
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,Switch Type: On/Off switch Maximum Wattage (pe...


In [58]:
stop_words = stopwords.words('english')

In [59]:
df_new['combined_features'] = df_new['combined_features'].str.lower().str.split()

In [60]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,"[switch, type:, on/off, switch, short, on, spa..."
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,"[switch, type:, foot, switch, maximum, wattage..."
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,"[switch, type:, rotary, socket, maximum, watta..."
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,"[switch, type:, 4-way, maximum, wattage, (per,..."
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,"[switch, type:, on/off, switch, maximum, watta..."


In [61]:
df_new["features"]=df_new["combined_features"].apply(lambda x: [word for word in x if word not in stop_words])

In [62]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features,features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,"[switch, type:, on/off, switch, short, on, spa...","[switch, type:, on/off, switch, short, space?,..."
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,"[switch, type:, foot, switch, maximum, wattage...","[switch, type:, foot, switch, maximum, wattage..."
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,"[switch, type:, rotary, socket, maximum, watta...","[switch, type:, rotary, socket, maximum, watta..."
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,"[switch, type:, 4-way, maximum, wattage, (per,...","[switch, type:, 4-way, maximum, wattage, (per,..."
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,"[switch, type:, on/off, switch, maximum, watta...","[switch, type:, on/off, switch, maximum, watta..."


In [63]:
df_new["features"]=df_new["features"].apply(lambda x: " ".join(x))

In [64]:
cv = CountVectorizer()

In [65]:
count_matrix = cv.fit_transform(df_new["features"])


In [66]:
count_matrix

<17209x13136 sparse matrix of type '<class 'numpy.int64'>'
	with 607000 stored elements in Compressed Sparse Row format>

In [67]:
cosine_sim = cosine_similarity(count_matrix)

In [68]:
product_user_likes = 'Sofa'

In [69]:
product_index = get_index_from_title(product_user_likes)

similar_products =  list(enumerate(cosine_sim[product_index]))

In [70]:
sorted_similar_products = sorted(similar_products,key=lambda x:x[1],reverse=True)

In [71]:
print('LIST OF ALL RECOMMENDED PRODUCT IF INPUT IS "{}"'.format(product_user_likes))
print()
print()
i=0
for element in sorted_similar_products:
        print (get_title_from_index(element[0]),get_home(element[0]))
        i=i+1
        if i>50:
            break

LIST OF ALL RECOMMENDED PRODUCT IF INPUT IS "Sofa"


Sofa https://www.wayfair.com/furniture/pdp/madison-home-usa-sofa-mhus1251.html
Mid Century Modern Ultra Plush Linen Fabric Sofa https://www.wayfair.com/furniture/pdp/madison-home-usa-mid-century-modern-ultra-plush-linen-fabric-sofa-mhus1163.html
Modern Sofa https://www.wayfair.com/furniture/pdp/madison-home-usa-modern-sofa-mhus1131.html
Elvie Sofa https://www.wayfair.com/furniture/pdp/ebern-designs-elvie-sofa-w000156164.html
Loveseat https://www.wayfair.com/furniture/pdp/madison-home-usa-loveseat-mhus1164.html
Sofa https://www.wayfair.com/furniture/pdp/madison-home-usa-sofa-mhus1202.html
Mid-Century Modern Large Sofa with Casters https://www.wayfair.com/furniture/pdp/madison-home-usa-mid-century-modern-large-sofa-with-casters-mhus1130.html
Ladner Sofa https://www.wayfair.com/furniture/pdp/wrought-studio-ladner-sofa-w000460892.html
Classic Plush Sofa https://www.wayfair.com/furniture/pdp/madison-home-usa-classic-plush-sofa-mhus1168.ht